<a href="https://colab.research.google.com/github/luthra2059/Sentiment-Analysis-with-scikit-learn/blob/master/Perform_Sentiment_Analysis_with_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('drive/My Drive/datasets/Sentimental Analysis/movie_data.csv')

In [4]:
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [5]:
df['review'][0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [6]:
import numpy as np

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
count = CountVectorizer()

In [15]:
docs = np.array(['The sun is shining', 'The weather is sweet', 'The sun is shining, the weather is sweet, and one and one is two'])

In [20]:
print(docs)
bag = count.fit_transform(docs)
print(count.vocabulary_)

['The sun is shining' 'The weather is sweet'
 'The sun is shining, the weather is sweet, and one and one is two']
{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [21]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [28]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)

In [29]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

In [31]:
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [32]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [33]:
import re

In [41]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [42]:
preprocessor(df.loc[0, 'review'][-50])

'i'

In [43]:
preprocessor("<a>this :) is a :( test :-)!")

'this is a test :) :( :)'

In [44]:
df['review'].apply(preprocessor)

0        in 1974 the teenager martha moxley maggie grac...
1        ok so i really like kris kristofferson and his...
2         spoiler do not read this if you think about w...
3        hi for all the people who have seen this wonde...
4        i recently bought the dvd forgetting just how ...
                               ...                        
49995    ok lets start with the best the building altho...
49996    the british heritage film industry is out of c...
49997    i don t even know where to begin on this one i...
49998    richard tyler is a little boy who is scared of...
49999    i waited long to watch this movie also because...
Name: review, Length: 50000, dtype: object

In [46]:
from nltk.stem.porter import PorterStemmer

In [47]:
porter = PorterStemmer()

In [48]:
def tokenizer(text):
  return text.split()

In [49]:
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [52]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [53]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [56]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [58]:
from nltk.corpus import stopwords

In [59]:
stop = stopwords.words('english')

In [61]:
[w for w in tokenizer_porter('a running like running and runs a lot')[-10:] if w not in stop]

['run', 'like', 'run', 'run', 'lot']

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, tokenizer=tokenizer_porter, use_idf=True, norm='l2', smooth_idf=True)

In [64]:
y = df.sentiment.values
x = tfidf.fit_transform(df.review)

In [68]:
from sklearn.model_selection import train_test_split

In [71]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5, shuffle=False)

In [72]:
import pickle

In [73]:
from sklearn.linear_model import LogisticRegressionCV

In [74]:
clf = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1, verbose=3, max_iter=300).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  6.6min finished


In [75]:
saved_model = open('saved_model.sav', 'wb')
pickle.dump(clf, saved_model)
saved_model.close()

In [76]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [77]:
saved_clf.score(X_test, y_test)

0.8892